In [1]:
#from utils import *

import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import re
from os import mkdir
from os.path import join, isfile, isdir, exists
import bcolz
import pickle 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
#from pattern.en import spelling
from tqdm import tqdm
import ast

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload

In [3]:
GPU = True
device_idx = 0
if GPU:
    device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


## Word Embedings : GloVe
This Class Loads the GloVe Embeding, processes it, and create a word embedding given the DataLoader.

In [4]:
class GloVe_embedding(object):
    def __init__(self,dim_vect = 25 ):
        ########## VARIABLES ##########
        self.dim_vect = dim_vect

        # Defining variables for GloVe: 
        self.words = []
        self.word2idx = {}
        self.glove_dict = {}
        
        ########## LOADING GLOVE DATA ##########
        
        # Defining path for GloVe Data : 
        self.path = join('..','data','glove') # Path of glove
        self.path_glove = join(self.path,'glove.twitter.27B.'+str(dim_vect))
        if not(isdir(self.path_glove)):
            mkdir(self.path_glove)
        self.path_vec_original = join(self.path,'glove.twitter.27B.'+str(dim_vect)+'d.txt') # Path of glove original vectors
        self.path_vec_save = join(self.path_glove,'glove.twitter.27B.'+str(dim_vect)+'d.vectors.dat')  # Path of glove saved vectors
        self.path_words = join(self.path_glove,'glove.twitter.27B.'+str(dim_vect)+'d.words.pkl')
        self.path_word2idx = join(self.path_glove,'glove.twitter.27B.'+str(dim_vect)+'d.word2idx.pkl')
                
        if not(isdir(self.path_vec_save) and isfile(self.path_words) and isfile(self.path_word2idx)) : 
            # If files are allready processed, just load them
            print('---- Processing the GloVe files : ',end='')
            self.process_GloVe()
            print('Done')
            
        # Load the wordvec files
        print('---- Loading the processed GloVe files : ',end='')
        self.load_GloVe()
        print('Done')
        
        ########## TORCH EMBEDDING ##########
        
        # Defining variables for our Embedding:
        self.size_vocab = len(self.words)
        
        # Creating the Pytorch Embedding Layer : 
        print('---- Creating the Pytorch Embedding Layer  : ',end='')
        self.emb_layer = nn.Embedding(self.size_vocab, self.dim_vect)
        self.create_emb_layer(non_trainable=True)
        print('Done')

               
    def process_GloVe(self):
        ''' Processes the GloVe Dataset - Saves files'''
        words = []
        word2idx = {}
        
        vectors = bcolz.carray(np.zeros(1) , rootdir=self.path_vec_save , mode='w' ) # defining vector saved
        
        # Adding Padding vector : 
        word2idx['<pad>'] = 0
        words.append('<pad>')
        #vect = np.random.normal(scale=0.6, size=(self.dim_vect , )) # random padding vect
        vect = np.zeros((self.dim_vect , )) # 0's padding vect. 
        vectors.append(vect)
        
        idx = 1
        with open(self.path_vec_original, 'rb') as f:
            for l in f:
                line = l.decode().split()
                word = line[0]
                words.append(word)
                word2idx[word] = idx
                idx += 1
                vect = np.array(line[1:]).astype(np.float)
                vectors.append(vect)
                

        vectors = bcolz.carray(vectors[:].reshape((-1, self.dim_vect)), rootdir=self.path_vec_save, mode='w')

        vectors.flush()
        pickle.dump(words, open(self.path_words, 'wb'))
        pickle.dump(word2idx, open(self.path_word2idx, 'wb'))
        
    def load_GloVe(self):
        ''' Loads previously processed dataset'''
        
        vectors = bcolz.open(self.path_vec_save)[:]
        
        self.words = pickle.load(open(self.path_words, 'rb'))
        self.word2idx = pickle.load(open(self.path_word2idx, 'rb'))
        
        self.glove_dict = {w: vectors[self.word2idx[w]] for w in self.words}
        self.emb_matrix = torch.Tensor(vectors)
    
    def create_emb_layer(self, non_trainable=True):
        self.emb_layer.load_state_dict({'weight': self.emb_matrix})
        if non_trainable:
            self.emb_layer.weight.requires_grad = False
        


In [5]:
myEmbedding = GloVe_embedding(dim_vect=200)

---- Loading the processed GloVe files : Done
---- Creating the Pytorch Embedding Layer  : Done


## Data Loader
This Class Loads the Tweet Dataset, Cleans it. It also enables the loading for the training and testing. 


In [6]:
class TestTweetDataset(Dataset):
    ''' 
    Pytorch Dataset for the Test set. 
    initialisation : - data : training pandas dataframe
                     - subtask : subtask we are working on {'subtask_a', 'subtask_b', 'subtask_c', }
                     - balanced : if we balance the dataset by oversampling it in the smallest classes
    '''
    def __init__(self,data, subtask):
        self.id = data.index.tolist()
        self.token = data.token.tolist()
        self.token_id = data.token_id.tolist()
        
    def __getitem__(self, index):
        return torch.LongTensor(self.token_id[index]), torch.FloatTensor([self.id[index]])

    def __len__(self):
        return len(self.token) 

        
    
class TweetDataset(Dataset):
    ''' 
    Pytorch Dataset for the Training set. 
    initialisation : - data : training pandas dataframe
                     - subtask : subtask we are working on {'subtask_a', 'subtask_b', 'subtask_c', }
                     - balanced : if we balance the dataset by oversampling it in the smallest classes
    '''
    def __init__(self,data,subtask):        
        # Save in lists the ids, labels, label_id, token, and token_id . 
        self.id = data.index.tolist()
        self.label_id = data[subtask].tolist()
        self.token = data.token.tolist()
        self.token_id = data.token_id.tolist()
        
    def __getitem__(self, index):
        return torch.LongTensor(self.token_id[index]), torch.FloatTensor([self.label_id[index]])

    def __len__(self):
        return len(self.token) 
    
    

In [7]:
class DataHandling(object):
    def __init__(self, embedding, pValid):
        print('-- Data Handling : ')
        
        self.embedding = embedding
        
        self.defineClasses()

        # All the Text Data path
        self.definePath()
        
        self.data = {}
        
        processed_ = True
        for f in self.path:
            processed_ = processed_ and isfile(self.path_clean[f])
        
        if  not(processed_) : 
            ### PROCESSING OF THE ORIGINAL DATASET
            # Load, Clean and Tokenize the Datasets
            print('---- Load, Clean and Tokensize Dataset : ',end='')
            self.inital_dataload()
            print('Done')
            
            # Compute List of All words in the datasets
            print('---- Finalize tokenized words and translation to id : ',end='')
            self.compute_wordlist()
            self.token2id()
            print('Done')

            # Add Embedding and correct clean the words not in embedding : 
            print('---- Adapt Dataset for Embedding : ',end='')
            self.adaptDataset()
            print('Done')

            # Save the Cleaned Datasets
            print('---- Saving all tokenized words : ',end='')
            self.save_cleanDataset()
            print('Done')
        else : 
            # Save the Cleaned Datasets
            print('---- Load the Clean Adapted Dataset : ',end='')
            self.load_cleanDataset()
            
            # Compute List of All words in the datasets
            self.compute_wordlist()
            print('Done')
        
        # Create Validation Set (split the test dataset) for every subtask
        self.splitValidation(p=pValid)
        self.prepareLabels()
        
        
    def defineClasses(self):
        ''' Function that defines the classes labels and id per subtask '''
        self.classes_dict = {}
        self.classes_dict['subtask_a'] = {'NOT' : 0 ,'OFF' : 1}
        self.classes_dict['subtask_b'] = {'UNT' : 0 ,'TIN' : 1}
        self.classes_dict['subtask_c'] = {'IND' : 0 ,'OTH' : 1, 'GRP' : 2}
        
    def definePath(self):
        ''' Function that defines all the paths of the datasets. '''
        self.path = {}
        self.path_clean = {}
        
        self.path['train'] = join('..','data','start-kit','training-v1','offenseval-training-v1.tsv')
        self.path_clean['train'] = join('..','data','start-kit','training-v1','clean-offenseval-training-v1.tsv')
        
        self.path['subtask_a'] = join('..','data','Test A Release','testset-taska.tsv')
        self.path_clean['subtask_a'] = join('..','data','Test A Release','clean-testset-taska.tsv')
        
        self.path['subtask_b'] = join('..','data','Test B Release','testset-taskb.tsv')
        self.path_clean['subtask_b'] = join('..','data','Test B Release','clean-testset-taskb.tsv')
        
        self.path['subtask_c'] = join('..','data','Test C Release','test_set_taskc.tsv')
        self.path_clean['subtask_c'] = join('..','data','Test C Release','clean-test_set_taskc.tsv')
        
    def getDataset(self, dataT='train',subtask='subtask_a',balanced = True):
        ''' Returns the pytorch Dataset
            - file : {'train','test','validation'}
            - subtask : {'subtask_a','subtask_b','subtask_c'} '''
        
            
        if dataT == 'train':
            if balanced : 
                data_train = self.balanceData(self.data[dataT][subtask],subtask)
            else : 
                data_train = self.data[dataT][subtask]
            dataset = TweetDataset(data_train, subtask)
        elif dataT == 'validation':
            dataset = TweetDataset(self.data[dataT][subtask], subtask)
        elif dataT == 'test':
            dataset = TestTweetDataset(self.data[subtask], subtask)
            
        return dataset
    
    def token2id(self):
        ''' Function that translates the list of tokens into a list of token id of the embedding.
            Adds a new 'token_id' column to the dataframe '''
        for f in self.path : 
            def token2id_x(x):
                
                return [self.embedding.word2idx[k] for k in x if k in self.embedding.words]
            self.data[f]['token_id'] = self.data[f]['token'].apply(lambda x : token2id_x(x))

    def save_cleanDataset(self):
        ''' Saves at the defined path the cleaned dataset '''
        for f in self.path : 
            self.data[f].to_csv(self.path_clean[f])
        
    def load_cleanDataset(self):
        ''' Loads at the defined path the cleaned dataset '''
        for f in self.path : 
            self.data[f] = pd.read_csv(self.path_clean[f],index_col='id')
            self.data[f]['token'] = self.data[f]['token'].apply(lambda x : ast.literal_eval(x))
            self.data[f]['token_id'] = self.data[f]['token_id'].apply(lambda x : ast.literal_eval(x))
           
               
    def adaptDataset(self):
        ''' Function that finds all the words which are not in the embedding and tries to 
            correct them with the pattern.en package by taking the most probable replacement.
            If the suggested word in very unlikely, the word is removed from the tweets. 
        '''
        # Find all words wich are not in the Embedding :
        missing_words = []
        for i, word in enumerate(self.all_words) :
            if self.embedding.word2idx.get(word) == None : 
                missing_words.append(word)
        
        # Correct if possible the missing_words : 
        ### We use theshold over which we correct the word. Under which we discard the word
        t = 0.5 # threshold
        rejected_words = []
        corrected_words = {}
        for word in tqdm(missing_words) : 
            suggestion, prob = spelling.suggest(word)[0]
            if prob < t : 
                rejected_words.append(word)
            else : 
                corrected_words[word] = suggestion
        
        # Modify the Original Datasets with those corrected_words : 
        for f in self.path : 
            self.data[f]['token'] = self.data[f]['token'].apply(lambda x : [corrected_words.get(k,k) for k in x])
            self.data[f]['token'] = self.data[f]['token'].apply(lambda x : [k for k in x if k not in rejected_words ])
        nb_rejected = len(rejected_words)
        nb_corrected = len(corrected_words)
        nb_vocab = len(self.embedding.glove_dict)
        p_rejected = 100* nb_rejected / nb_vocab
        p_corrected = 100* nb_corrected / nb_vocab
        print('---- Words removed   : {0:} / {1:.2f} - {2:} %'.format(nb_rejected,nb_vocab,p_rejected))
        print('---- Words corrected : {0:} / {1:.2f} - {2:} %'.format(nb_corrected,nb_vocab,p_corrected))
        
    def inital_dataload(self):
        for f in self.path : 
            self.data[f] = pd.read_table(self.path[f],index_col='id')
            self.data[f]['token'] = self.data[f]['tweet'].apply(lambda x : self.clean_tweet(x))
            
    def compute_wordlist(self):
        self.all_words_freq = {}
        self.all_words = []
        
        for f in self.data : 
            for i in range(len(self.data[f])):
                for e in self.data[f].iloc[i].token:
                    self.all_words_freq[e] = 1 + self.all_words_freq.get(e,0)
        self.all_words = list(self.all_words_freq.keys())
        
    def splitValidation(self,p):
        ''' Creates the validation set by  taking p % of the train dataset '''
        data = self.data['train'].copy()
        self.data['train'] = {}
        self.data['validation'] = {}

        for subtask in self.classes_dict: # per subtask
            self.data['train'][subtask] = pd.DataFrame()
            self.data['validation'][subtask]= pd.DataFrame()
            for label in self.classes_dict[subtask]: #per label in this subtask 
                data_label =  data[data[subtask]==label]
                self.data['train'][subtask] = self.data['train'][subtask].append(data.loc[data_label.index])
                nb_valid = int(len(data_label)*p)
                # Select randmoly (without repetition) the indexes of the selected vaidation tweets
                index_valid = np.random.choice(data_label.index,(nb_valid,),replace=False)
                # Add the the selected validation tweets to the new dataframe
                self.data['validation'][subtask] = self.data['validation'][subtask].append(self.data['train'][subtask].loc[index_valid,:])
                # Drop the selected validation tweets from the training set
                self.data['train'][subtask] = self.data['train'][subtask].drop(index = index_valid)
                
    def prepareLabels(self) : 
        ''' Transform the labels into classes id '''
        for subtask in self.classes_dict: # per subtask
            self.data['validation'][subtask][subtask] =self.data['validation'][subtask][subtask].apply(lambda x : self.classes_dict[subtask][x])  
            self.data['train'][subtask][subtask] = self.data['train'][subtask][subtask].apply(lambda x : self.classes_dict[subtask][x])  

    def balanceData(self,data,subtask):
        ''' Augments the Data given in input in order to balance the dataset'''
        class_size = {}
        for label in self.classes_dict[subtask]:
            class_size[label] = len(data[data[subtask]==self.classes_dict[subtask][label]])
        largest_class = max(class_size, key=class_size.get)
        print('---- Augmenting the Data : ')
        print('Before Augmentation : ',class_size)

        for label in self.classes_dict[subtask]:  
            if label != largest_class:
                id_list = data[data[subtask]==self.classes_dict[subtask][label]].index
                nb_augmentation = class_size[largest_class] - class_size[label]
                id_augmentation = np.random.choice(id_list, (nb_augmentation,))
                data = data.append(data.loc[id_augmentation,:])
        # Check if it went well
        for label in self.classes_dict[subtask]:
            class_size[label] = len(data[data[subtask]==self.classes_dict[subtask][label]])
        
        print('After Augmentation : ',class_size)
        return data
    
    def clean_tweet(self,text):
        ''' Function that is applied to every to tweet in the dataset '''
        
        # =========== TEXT ===========
        # Replace @USER by <user>
        text = re.compile(r'@USER').sub(r'<user>',text)

        # Replace URL by <url>
        text = re.compile(r'URL').sub(r'<url>',text)

        # Remove numbers :
        text = re.compile(r'[0-9]+').sub(r' ',text)

        # Remove some special characters
        text = re.compile(r'([\xa0_\{\}\[\]¬•$,:;/@#|\^*%().~`”"“-])').sub(r' ',text) 

        # Space the special characters with white spaces
        text = re.compile(r'([$&+,:;=?@#|\'.^*()%!"’“-])').sub(r' \1 ',text)
        
        # Replace some special characters : 
        replace_dict = {r'&' : 'and' , 
                        r'\+' : 'plus'}
        for cha in replace_dict:
            text = re.compile(str(cha)).sub(str(replace_dict[cha]),text)
            
        # Handle Emoji : translate some and delete the others
        text = self.handle_emoji(text)
        
        # Word delengthening : 
        text = re.compile(r'(.)\1{3,}').sub(r'\1\1',text)

        # Cut the words with caps in them : 
        text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)
        text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)        
        # =========== TOKENS ===========
        # TOKENIZE 
        text = text.split(' ')

        # Remove white spaces tokens
        text = [text[i] for i in range(len(text)) if text[i] != ' ']

        # Remove empty tokens
        text = [text[i] for i in range(len(text)) if text[i] != '']

        # Remove repetition in tokens (!!! => !)
        text = [text[i] for i in range(len(text)) if text[i] != text[i-1]]

        #  Handle the ALL CAPS Tweets 
        ### if ratio of caps in the word > 75% add allcaps tag <allcaps>
        caps_r = np.mean([text[i].isupper() for i in range(len(text))])
        if caps_r > 0.6 : 
            text.append('<allcaps>')

        # Lower Case : 
        text = [text[i].lower() for i in range(len(text))]

        return text

    def handle_emoji(self,text):
        # Dictionnary of "important" emojis : 
        emoji_dict =  {'♥️': ' love ',
                       '❤️' : ' love ',
                       '❤' : ' love ',
                       '😘' : ' kisses ',
                      '😭' : ' cry ',
                      '💪' : ' strong ',
                      '🌍' : ' earth ',
                      '💰' : ' money ',
                      '👍' : ' ok ',
                       '👌' : ' ok ',
                      '😡' : ' angry ',
                      '🍆' : ' dick ',
                      '🤣' : ' haha ',
                      '😂' : ' haha ',
                      '🖕' : ' fuck you '}

        for cha in emoji_dict:
            text = re.compile(str(cha)).sub(str(emoji_dict[cha]),text)
        # Remove ALL emojis
        text = emoji.get_emoji_regexp().sub(r' ',text) 
        text = re.compile("([\U0001f3fb-\U0001f3ff])").sub(r'',text) 
        text = re.compile("([\U00010000-\U0010ffff])").sub(r'',text) 
        text = re.compile("(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])").sub(r'',text)

        # Add Space between  the Emoji Expressions : 
        text = re.compile("([\U00010000-\U0010ffff])").sub(r' \1 ',text) 
        return text

In [8]:
mydata = DataHandling(myEmbedding, pValid=0.15)

-- Data Handling : 
---- Load the Clean Adapted Dataset : Done


In [9]:
mydata.data['train']['subtask_a'].head(10)

,tweet,subtask_a,subtask_b,subtask_c,token,token_id
id,,,,,,
16820,Amazon is investigating Chinese employees who ...,0,NaN,NaN,"[amazon, is, investigating, chinese, employees...","[2547, 33, 39459, 3492, 16270, 128, 71, 5442, ..."
43605,@USER @USER Obama wanted liberals &amp; illega...,0,NaN,NaN,"[<user>, obama, wanted, liberals, and, amp, il...","[1, 1382, 953, 30168, 27, 12801, 114750, 17, 9..."
45157,@USER Buy more icecream!!!,0,NaN,NaN,"[<user>, buy, more, icecream, !]","[1, 873, 146, 18658, 10]"
42992,@USER What’s the difference between #Kavanaugh...,0,NaN,NaN,"[<user>, what, s, the, difference, between, an...","[1, 87, 138, 14, 2954, 1472, 27, 1, 97, 40, 37..."
54920,@USER @USER @USER It should scare every Americ...,0,NaN,NaN,"[<user>, it, should, scare, every, american, !...","[1, 34, 277, 9603, 382, 2034, 10, 148, 33, 701..."
56392,@USER @USER @USER @USER @USER @USER @USER @USE...,0,NaN,NaN,"[<user>, i, like, my, soda, like, i, like, my,...","[1, 11, 64, 30, 9477, 64, 11, 64, 30, 290103, ..."
86735,@USER you are also the king of taste,0,NaN,NaN,"[<user>, you, are, also, the, king, of, taste]","[1, 16, 71, 895, 14, 1697, 40, 3212]"
71446,5/5: @USER The time is right for this House to...,0,NaN,NaN,"[<user>, the, time, is, right, for, this, hous...","[1, 14, 136, 33, 209, 38, 54, 544, 17, 7094, 1..."
23958,@USER Besides Jax’s mom and maybe Ope he is ha...,0,NaN,NaN,"[<user>, besides, jax, s, mom, and, maybe, ope...","[1, 8101, 43627, 138, 570, 27, 718, 50199, 108..."


In [10]:
# Example of Tweet Processing
print(mydata.data['train']['subtask_a'].iloc[331]['tweet'])
print(mydata.data['train']['subtask_a'].iloc[331]['token'])
print(mydata.data['train']['subtask_a'].iloc[331]['token_id'])

@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER You are suggesting that it is not the     law and policy (sanctions regime) that informs rates of benefit sanctions. The graph would suggest that is true as by 2017 the sanctions rate was lower than it was under the previous sanctions regime. Good observations about pressure"++"
['<user>', 'you', 'are', 'suggesting', 'that', 'it', 'is', 'not', 'the', 'law', 'and', 'policy', 'sanctions', 'regime', 'that', 'informs', 'rates', 'of', 'benefit', 'sanctions', 'the', 'graph', 'would', 'suggest', 'that', 'is', 'true', 'as', 'by', 'the', 'sanctions', 'rate', 'was', 'lower', 'than', 'it', 'was', 'under', 'the', 'previous', 'sanctions', 'regime', 'good', 'observations', 'about', 'pressure', 'plus']
[1, 16, 71, 49077, 46, 34, 33, 79, 14, 3188, 27, 10185, 41954, 22981, 46, 163098, 12821, 40, 11412, 41954, 14, 34884, 197, 11276, 46, 33, 548, 125, 153, 14, 41954, 3761, 94, 7671, 286, 34, 94, 14

## Classifier 
Set of Classes used as classifier for the tweets. 

In [11]:
# Classification NN : 
class FFNN(nn.Module):
    
    def __init__(self, embedding, hidden_dim , num_classes ,embedding_dim):
        print('------ Creating FFNN : ',end='')
        
        super(FFNN, self).__init__()
        
        # Embedding
        self.embedding = embedding
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes) 
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

        # Activation Layers
        self.relu1 = nn.ReLU()

        self.output = nn.LogSoftmax(dim=1)
        print('Done')
    
    def forward(self, x):
        
        embedded = self.embedding(x)
        # we average the embeddings of words in a sentence
        
        non_zero_nb = (x!=0).sum(1,keepdim=True)
        #print(x.shape, non_zero_nb,embedded.sum(1).shape)
        averaged = embedded.sum(1) / non_zero_nb.float()
        #averaged = embedded.mean(1)
        # (batch size, max sent length, embedding dim) to (batch size, embedding dim)

        out = self.fc1(averaged)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.output(out)
        return out
    
    def loss_fn(self):
        ''' Returns the loss function best associated with the model'''
        return nn.NLLLoss()


In [12]:
# CNN : 
class CNN(nn.Module):
    
    def __init__(self, embedding, embedding_dim, out_channels, window_size, output_dim, dropout):
        
        super(CNN, self).__init__()
        
        self.embedding = embedding
        
        #in_channels -- 1 text channel
        #out_channels -- the number of output channels
        #kernel_size is (window size x embedding dim)
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size,embedding_dim))
        
        #the dropout layer
        self.dropout = nn.Dropout(dropout)
    
        #the output layer
        self.fc = nn.Linear(out_channels, output_dim)
        
        self.out =  nn.LogSoftmax(dim=1)
        
    def forward(self, x):
                
        #(batch size, max sent length)
        
        embedded = self.embedding(x)
        #print('embedded',embedded.shape)
                
        #(batch size, max sent length, embedding dim)
        
        #images have 3 RGB channels 
        #for the text we add 1 channel
        embedded = embedded.unsqueeze(1)
        #print('embedded_unsqueeze',embedded.shape)
        
        #(batch size, 1, max sent length, embedding dim)
        
        feature_maps = self.conv(embedded)
        #print('feature_maps',feature_maps.shape)

        #??? what is the shape of the convolution output
        
        #(batch size, n filters, max input length - window size +1, 1)
        
        feature_maps = feature_maps.squeeze(3)
        #print('feature_maps_unsqueeze',feature_maps.shape)

        #??? why do we reduce 1 dimention here
        
        # we do need the 1 channel anymore
                
        feature_maps = F.relu(feature_maps)
        #print('feature_maps_relu',feature_maps.shape)

  
        #the max pooling layer
        pooled = F.max_pool1d(feature_maps, feature_maps.shape[2])
        #print('pooled',pooled.shape)

        pooled = pooled.squeeze(2)
        #print('pooled_squeeze',pooled.shape)

        #??? what is the shape of the pooling output
        #(batch size, n_filters)
        
        dropped = self.dropout(pooled)
        #print('dropped',dropped.shape)

        preds = self.fc(dropped)
        #print('preds', preds.shape)
        out = self.out(preds)
        #print('out',out.shape)
        
        return out
    
    def loss_fn(self):
        return nn.NLLLoss()

In [13]:
class CNN_paper(nn.Module):
    '''based on https://arxiv.org/pdf/1408.5882'''
    def __init__(self, embedding, embedding_dim, out_channels, window_size, output_dim, dropout):
        
        super(CNN_paper, self).__init__()
        
        self.embedding = embedding

        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size[0],embedding_dim))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size[1],embedding_dim))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size[2],embedding_dim))
        #the dropout layer
        self.dropout = nn.Dropout(dropout)
    
        #the output layer
        self.fc = nn.Linear(out_channels*3, output_dim)
        
        self.out =  nn.LogSoftmax(dim=1)
    
    def conv_block(self,embedded,conv_layer):
        feature_maps = conv_layer(embedded)
        features_maps = feature_maps.squeeze(3)
        feature_maps = F.relu(feature_maps)
        return F.max_pool1d(feature_maps, feature_maps.shape[2]).squeeze(2)
    
    def forward(self, x):
                
        #(batch size, max sent length)
        
        embedded = self.embedding(x)
                        
        #images have 3 RGB channels 
        #for the text we add 1 channel
        embedded = embedded.unsqueeze(1)
         

        feature_maps1 = self.conv1(embedded)
        feature_maps2 = self.conv2(embedded)
        feature_maps3 = self.conv3(embedded)
        
        feature_maps1 = feature_maps1.squeeze(3)
        feature_maps2 = feature_maps2.squeeze(3)
        feature_maps3 = feature_maps3.squeeze(3)


                
        feature_maps1 = F.relu(feature_maps1)
        feature_maps2 = F.relu(feature_maps2)
        feature_maps3 = F.relu(feature_maps3)

  
        #the max pooling layer
        pooled1 = F.max_pool1d(feature_maps1, feature_maps1.shape[2])
        pooled2 = F.max_pool1d(feature_maps2, feature_maps2.shape[2])
        pooled3 = F.max_pool1d(feature_maps3, feature_maps3.shape[2])

        pooled1 = pooled1.squeeze(2)
        pooled2 = pooled2.squeeze(2)
        pooled3 = pooled3.squeeze(2)
        all_pooled =torch.cat((pooled1, pooled2, pooled3), 1)
        
        dropped = self.dropout(all_pooled)

        preds = self.fc(dropped)
        out = self.out(preds)
        
        return out
    
    def loss_fn(self):
        return nn.NLLLoss()

In [14]:
# RNN, GRU and LSTM
class RNN_base(nn.Module):
    def __init__(self, embedding, embedding_dim=25, hidden_size=25, output_size=2, num_layers=1, output_layer=None):
        super(RNN_base, self).__init__()
        
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        if output_layer==None:
            self.i2o = nn.Linear(hidden_size, output_size)
        else:
            self.i2o = output_layer
        self.final_activation = nn.LogSoftmax(dim=1)
        self.final_activation = nn.LogSoftmax(dim=1)
        
        
    def forward(self, inputs):
        inputs_embedded = self.embedding(inputs).float()
 
        sentence_lengths = (inputs_embedded.sum(dim=2,keepdim=False)!=0).sum(dim=1)
        
        output, hidden = self.rnn(inputs_embedded)
        
        output_final = torch.empty(inputs.shape[0],self.hidden_size).float()
        output_final = output[torch.arange(inputs.shape[0]),sentence_lengths-1,:]

        output = self.i2o(output_final)
        output = self.final_activation(output)
        
        return output

    def loss_fn(self):
        ''' Returns the loss function best associated with the model'''
        return nn.NLLLoss()
    
class RNN(RNN_base):
    def __init__(self, embedding, embedding_dim=25, hidden_size=25, output_size=2, num_layers=1, output_layer=None):
        RNN_base.__init__(self, 
                          embedding=embedding, 
                          embedding_dim=embedding_dim, 
                          hidden_size=hidden_size, 
                          output_size=output_size, 
                          num_layers=num_layers, 
                          output_layer=output_layer)
        self.rnn = nn.RNN(
            input_size = embedding_dim, 
            hidden_size = hidden_size,
            num_layers = num_layers,
            nonlinearity = "relu",
            batch_first = True
        )

class LSTM(RNN_base):
    def __init__(self, embedding, embedding_dim=25, hidden_size=25, output_size=2, num_layers=1, output_layer=None):
        RNN_base.__init__(self, 
                          embedding=embedding, 
                          embedding_dim=embedding_dim, 
                          hidden_size=hidden_size, 
                          output_size=output_size, 
                          num_layers=num_layers, 
                          output_layer=output_layer)
        self.rnn = nn.LSTM(
            input_size = embedding_dim, 
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True
        )  
        
class GRU(RNN_base):
    def __init__(self, embedding, embedding_dim=25, hidden_size=25, output_size=2, num_layers=1, output_layer=None):
        RNN_base.__init__(self, 
                          embedding=embedding, 
                          embedding_dim=embedding_dim, 
                          hidden_size=hidden_size, 
                          output_size=output_size, 
                          num_layers=num_layers, 
                          output_layer=output_layer)
        self.rnn = nn.GRU(
            input_size = embedding_dim, 
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True
        ) 
        
def output_layer(input_size,output_size,middle_layers=[]):
    "Input and output size - scalars. Middle layers - list (ignore if just one layer wanted)"
    
    #For current implementation, the input_size should be equal to the size of the hidden layer for the RNNs.
    activation = nn.ReLU()

    component_layers=[]
    if middle_layers!=[]:

        component_layers.append(
            nn.Linear(
                in_features  = input_size,
                out_features  = middle_layers[0]))
        component_layers.append(activation)


        if len(middle_layers)>1:
            for layer in range(1,len(middle_layers)):
                component_layers.append(
                    nn.Linear(in_features  = middle_layers[layer-1],
                              out_features  = middle_layers[layer]))
                component_layers.append(activation)

        component_layers.append(
            nn.Linear(in_features  = middle_layers[-1],
                      out_features  = output_size))

    else:
        component_layers.append(
            nn.Linear(in_features  = input_size,
                      out_features  = output_size))

    i2o = nn.Sequential(*component_layers)
    
    return i2o

Pad Data Method : Used with the pytorch DataLoader in order to pad the length of the tweets by batch. 

In [15]:
class padding_tweet:
    '''
    Pad Data Method : Used with the pytorch DataLoader in order to pad the length of the tweets by batch. 
    args: 
        batch - List of elements ( x , label )
    return 
        batch - Padded ( list(x) , list(label))
    
    '''
    def __init__(self, max_len):
        self.max_len = max_len
    def __call__(self,batch):
        
        batch = list(zip(*batch))
       
        batch[0] = torch.stack([self.pad_tensor(vec=t, pad=self.max_len, dim=0) for t in batch[0]],dim=0)
        batch[1] = torch.stack(batch[1])
        return batch[0] , batch[1]

    def pad_tensor(self,vec, pad, dim):
        """
        args:
            vec - tensor to pad
            pad - the size to pad to
            dim - dimension to pad

        return:
            a new tensor padded to 'pad' in dimension 'dim'
        """
        pad_size = list(vec.shape)
        pad_size[dim] = pad - vec.size(dim)
        return torch.cat([vec, torch.zeros(*pad_size,dtype=torch.long)], dim=dim)


## Main Class : Trainer
Main Class for the loading, training, testing etc ...

In [29]:
class OffensiveClassifier(object):
    ''' Main Class for the loading, training, testing etc ...'''
    def __init__(self,subtask='subtask_a', dim_vect=25,pValid = 0.15):
        
        self.dim_vect = dim_vect
        
        self.subtask = subtask
        

        # Loading the GloVe Embedding and Torch Formating of this Embedding
        self.GloVe = GloVe_embedding(dim_vect= dim_vect )
        self.embedding = self.GloVe.emb_layer
        
        # Loading the Data Handler : 
        self.dataHandler = DataHandling(self.GloVe,pValid=pValid)
        
        # Retrieving Training DataSet (pytorch)
        self.train_set = self.dataHandler.getDataset('train',subtask,balanced=True)
        
        # Retrieving the Validation Set (pytorch)
        self.valid_set = self.dataHandler.getDataset('validation',subtask)

        # Retrieving Test DataSet (pytorch)
        self.test_set = self.dataHandler.getDataset('test',subtask)
        
        # Usefull info on classes : 
        self.class2id  = self.dataHandler.classes_dict[self.subtask]
        self.id2class = dict(zip(self.class2id.values(),self.class2id.keys()))
        self.nb_class = len(self.class2id)
        self.max_len  = max([ max([len(t) for t in self.train_set.token]),
                              max([len(t) for t in self.valid_set.token]),
                              max([len(t) for t in self.test_set.token])])
            
        
    
    def train( self, nb_epochs, optimizer , model , batch_size = 1000 ,  ):
        
        self.train_generator = DataLoader(self.train_set, batch_size=batch_size,collate_fn=padding_tweet(self.max_len), shuffle=True)

        self.model = model.to(device)
        loss_fn = self.model.loss_fn()

        for epoch in range(nb_epochs):
            i_batch = 0
            accuracy_average = 0
            self.model.train() 
            
            for tokens, target  in self.train_generator :
                i_batch += 1
                target = target.long().view((-1,)).to(device)
                tokens = tokens.long().to(device)
                #to ensure the dropout (exlained later) is "turned on" while training
                #good practice to include even if do not use here
                self.model.train()

                #we zero the gradients as they are not removed automatically
                optimizer.zero_grad()

                output = self.model(tokens)
                #print(output.shape)

                predictions = torch.argmax(output,dim=1).float()
                loss = loss_fn(output, target)
                acc, correct = self.accuracy(predictions, target)
                accuracy_average += acc
                #calculate the gradient of each parameter
                loss.backward()
                
                #update the parameters using the gradients and optimizer algorithm 
                optimizer.step()

                epoch_loss = loss.item()

                if i_batch % (nb_epochs -1 )  ==0:
                    pass
            print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.3f} | Train Acc: {accuracy_average*100/i_batch:.2f}%')
            f1 = self.validation(batch_size = 50)
            print('=============================================================================')
            print()
            #return f1
            
    def validation(self, batch_size = 1000):
        all_correct = 0
        self.validation_generator = DataLoader(self.valid_set, batch_size=batch_size, collate_fn=padding_tweet(self.max_len), shuffle=True)
        loss_fn = self.model.loss_fn()
        nb_valid = len(classifier.valid_set)
        self.model.eval()  # set model to evaluation mode
        all_prediction = []
        all_target = []
        
        with torch.no_grad(): 
            for tokens, target  in self.validation_generator :
                target = target.long().view((-1,)).to(device)
                tokens = tokens.long().to(device)
                
                output = self.model(tokens)
                predictions = torch.argmax(output,dim=1)
                
                loss = loss_fn(output, target)
                
                all_prediction.extend(predictions.view(-1,).tolist())
                all_target.extend(target.view(-1,).tolist())
                
        accuracy, CM, stats_df = self.results(all_prediction,all_target)
        print(f'| Stats on the Validation : ')
        print(stats_df)
        
        print(f'| Validation Accuracy : {100*accuracy:.2f} % - Macro F1 Score : {stats_df.loc["f1-measure"].mean():.4f} ')
        print(f'| CM on the Validation : ')
        print(CM)
        print()
        #return stats_df.loc["f1-measure"].mean()
        
    def test(self,folder):
        ''' 
            Test Function : Tests the Network on the Test Data of the Subtask and Saves in a file
        '''
        test_result = pd.DataFrame()
        self.test_generator = DataLoader(self.test_set,collate_fn= padding_tweet(self.max_len) )
        nb_valid = len(classifier.valid_set)
        self.model.eval()  # set model to evaluation mode
        all_prediction = []
        all_id = []
        with torch.no_grad(): 
            for tokens, id_tweet  in self.test_generator :
                id_tweet = id_tweet.long().view((-1,))
                tokens = tokens.long().to(device)
                
                output = self.model(tokens)
                predictions = torch.argmax(output,dim=1)
                
                all_prediction.extend(predictions.view(-1,).tolist())
                all_id.extend(id_tweet.view(-1,).tolist())
        results_df = pd.DataFrame(all_prediction, index=all_id)
              
        # Change id_labels to labels
        results_df[0] = results_df[0].apply(lambda x : self.id2class[x])
              
        # Saving the Results in the right format in the folder ../data/results/*folder*
        save_path = join('..','data','results')
        if not(exists(save_path)) : mkdir(save_path)
        save_path = join(save_path,folder)
        if not(exists(save_path)) : mkdir(save_path)
        
        path_file = join(save_path, self.subtask + '.csv')
        i = 0
        while exists(path_file):
              i += 1
              path_file = join(save_path, self.subtask + '_'+ str(i) +'.csv')
        results_df.to_csv(path_file, header=False)
        print('| Successfully save to : ', path_file)
        

    def accuracy(self, output, target):
        target = np.array(target.tolist()).astype(int)
        output = np.array(torch.round(output).tolist()).astype(int)
        correct = (output == target)
        accuracy = correct.sum()/len(correct)
        return accuracy, correct
        
    def results(self, output, target ):
        target = np.array(target).astype(int)
        output = np.round(output).astype(int)
        correct = (output == target)
        accuracy = correct.sum()/len(correct)
        
        CM = np.zeros((self.nb_class,self.nb_class))
        for i in range(len(output)): 
            CM[target[i], output[i]] += 1
        
        # compute the interesting stats on the classification : 
        stats_df = self.stats(CM)
        stats_df.columns = map(lambda x : self.id2class[x] ,stats_df.columns)
        return (accuracy, CM, stats_df)

    def stats(self,CM):   
        n_class = self.nb_class
    
        stats = {}
        stats['precision'] = {} 
        stats['recall'] = {} 
        stats['accuracy'] = {} 
        stats['f1-measure'] = {} 

        for t in range(self.nb_class):
            tp = CM[t,t]
            tn = np.sum([CM[i,i] for i in range(self.nb_class) if i != t])

            fp = np.sum([CM[i,t] for i in range(self.nb_class) if i != t])
            fn = np.sum([CM[t,i] for i in range(self.nb_class) if i != t])

            # compute accuracy per class :
            accuracy = (tp + tn) / (tp + tn + fp + fn)

            # compute recall per class
            recall = tp /(tp + fn)

            # compute precision per class 
            precision = tp / (tp + fp)

            # compute F1-measure per class 
            f1 = 2*tp / (2* tp + fp + fn)

            # saving stats : 

            stats['precision'][t] = precision
            stats['recall'][t] = recall
            stats['accuracy'][t] = accuracy
            stats['f1-measure'][t] = f1

        # Compute Average classification rate
        stats = pd.DataFrame(stats).transpose()
        return stats


## Evaluation of the Model 
Choice of the sub-task, the dimension of the embedding vectors, and the percentage of validation taken from train.

**Best choice** : `dim_vect = 200` and `pValid = 0.2`


In [150]:
# 1st step : Define Classifier for specific task
#classifier = OffensiveClassifier(subtask='subtask_a', dim_vect=200 , pValid = 0.2)
#classifier = OffensiveClassifier(subtask='subtask_b', dim_vect=200 , pValid = 0.2)
classifier = OffensiveClassifier(subtask='subtask_c', dim_vect=200 , pValid = 0.2)

---- Loading the processed GloVe files : Done
---- Creating the Pytorch Embedding Layer  : Done
-- Data Handling : 
---- Load the Clean Adapted Dataset : Done
---- Augmenting the Data : 
Before Augmentation :  {'IND': 1926, 'OTH': 316, 'GRP': 860}
After Augmentation :  {'IND': 1926, 'OTH': 1926, 'GRP': 1926}


### 1) FFNN
Best Parameters :

In [107]:
# 2nd Step : Define Model : 
model = FFNN(embedding= classifier.embedding, 
             hidden_dim= 100 ,
             num_classes = classifier.nb_class,
             embedding_dim= classifier.dim_vect)

------ Creating FFNN : Done


In [108]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.00001,weight_decay=0.005)

In [109]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 20,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 50)

| Epoch: 00 | Train Loss: 1.084 | Train Acc: 33.35%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.500000  0.101050  0.000000
recall      0.010395  0.974684  0.000000
accuracy    0.145648  0.106632  0.275168
f1-measure  0.020367  0.183115  0.000000
| Validation Accuracy : 10.59 % - Macro F1 Score : 0.0678 
| CM on the Validation : 
[[  5. 475.   1.]
 [  1.  77.   1.]
 [  4. 210.   0.]]


| Epoch: 01 | Train Loss: 1.167 | Train Acc: 33.11%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.533333  0.099338  0.000000
recall      0.016632  0.949367  0.000000
accuracy    0.147425  0.108214  0.275748
f1-measure  0.032258  0.179856  0.000000
| Validation Accuracy : 10.72 % - Macro F1 Score : 0.0707 
| CM on the Validation : 
[[  8. 471.   2.]
 [  2.  75.   2.]
 [  5. 209.   0.]]


| Epoch: 02 | Train Loss: 1.204 | Train Acc: 33.39%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.714286  

| Epoch: 17 | Train Loss: 1.059 | Train Acc: 51.38%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.849558  0.124378  0.495726
recall      0.598753  0.316456  0.542056
accuracy    0.637444  0.650986  0.665116
f1-measure  0.702439  0.178571  0.517857
| Validation Accuracy : 55.43 % - Macro F1 Score : 0.4663 
| CM on the Validation : 
[[288. 113.  80.]
 [ 16.  25.  38.]
 [ 35.  63. 116.]]


| Epoch: 18 | Train Loss: 1.006 | Train Acc: 52.03%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.856305  0.128205  0.508403
recall      0.607069  0.316456  0.565421
accuracy    0.647929  0.661631  0.675926
f1-measure  0.710462  0.182482  0.535398
| Validation Accuracy : 56.59 % - Macro F1 Score : 0.4761 
| CM on the Validation : 
[[292. 110.  79.]
 [ 16.  25.  38.]
 [ 33.  60. 121.]]


| Epoch: 19 | Train Loss: 1.039 | Train Acc: 51.75%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.857550  

In [110]:
# 5th Step : Test the Model and save data 
classifier.test(folder='FFNN')

| Successfully save to :  ../data/results/FFNN/subtask_c.csv


### 2) CNN : 1 Layer
Best Parameters :

In [151]:
# 2nd Step : Define Model : 
model = CNN( embedding= classifier.embedding, 
             embedding_dim = classifier.dim_vect,
             out_channels= 150,
             window_size= 3,
             output_dim= classifier.nb_class,
             dropout = 0.5)

In [152]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.00001,weight_decay=0.005)
#optimizer = optim.RMSprop(model.parameters(), lr=0.001,weight_decay=0.001)

In [153]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 50,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 20)

| Epoch: 00 | Train Loss: 1.044 | Train Acc: 34.57%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.888889  0.160000  0.283174
recall      0.033264  0.050633  0.967290
accuracy    0.327089  0.702786  0.299472
f1-measure  0.064128  0.076923  0.438095
| Validation Accuracy : 29.33 % - Macro F1 Score : 0.1930 
| CM on the Validation : 
[[ 16.  16. 449.]
 [  0.   4.  75.]
 [  2.   5. 207.]]


| Epoch: 01 | Train Loss: 1.232 | Train Acc: 37.00%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.861538  0.089888  0.329730
recall      0.232848  0.101266  0.855140
accuracy    0.439130  0.665934  0.429178
f1-measure  0.366612  0.095238  0.475943
| Validation Accuracy : 39.15 % - Macro F1 Score : 0.3126 
| CM on the Validation : 
[[112.  65. 304.]
 [  3.   8.  68.]
 [ 15.  16. 183.]]


| Epoch: 02 | Train Loss: 1.117 | Train Acc: 39.19%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.844898  

| Epoch: 17 | Train Loss: 0.861 | Train Acc: 59.33%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.822055  0.152381  0.511111
recall      0.681913  0.202532  0.644860
accuracy    0.682720  0.760252  0.698551
f1-measure  0.745455  0.173913  0.570248
| Validation Accuracy : 62.27 % - Macro F1 Score : 0.4965 
| CM on the Validation : 
[[328.  56.  97.]
 [ 28.  16.  35.]
 [ 43.  33. 138.]]


| Epoch: 18 | Train Loss: 0.916 | Train Acc: 59.66%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.822500  0.144144  0.513308
recall      0.683992  0.202532  0.630841
accuracy    0.682788  0.752351  0.698690
f1-measure  0.746879  0.168421  0.566038
| Validation Accuracy : 62.02 % - Macro F1 Score : 0.4938 
| CM on the Validation : 
[[329.  59.  93.]
 [ 28.  16.  35.]
 [ 43.  36. 135.]]


| Epoch: 19 | Train Loss: 0.979 | Train Acc: 59.92%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.824121  

| Epoch: 34 | Train Loss: 0.724 | Train Acc: 66.44%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.827251  0.175000  0.534979
recall      0.706861  0.265823  0.607477
accuracy    0.698435  0.757716  0.713663
f1-measure  0.762332  0.211055  0.568928
| Validation Accuracy : 63.44 % - Macro F1 Score : 0.5141 
| CM on the Validation : 
[[340.  59.  82.]
 [ 27.  21.  31.]
 [ 44.  40. 130.]]


| Epoch: 35 | Train Loss: 0.795 | Train Acc: 66.14%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.826829  0.173554  0.534979
recall      0.704782  0.265823  0.607477
accuracy    0.697013  0.756173  0.713246
f1-measure  0.760943  0.210000  0.568928
| Validation Accuracy : 63.31 % - Macro F1 Score : 0.5133 
| CM on the Validation : 
[[339.  60.  82.]
 [ 27.  21.  31.]
 [ 44.  40. 130.]]


| Epoch: 36 | Train Loss: 1.053 | Train Acc: 66.83%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.828851  

In [154]:
# 5th Step : Test the Model and save data 
classifier.test(folder='CNN')

| Successfully save to :  ../data/results/CNN/subtask_c.csv


### 3) CNN : 3 Layers
Best Parameters :
CNN_paper : nb epoch 3, batch_size= 20, best f1 for A : 0.75 optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.005)

In [173]:
# 2nd Step : Define Model : 
model = CNN_paper(embedding= classifier.embedding, 
             embedding_dim = classifier.dim_vect,
             out_channels= 100,
             window_size= [3,4,5],
             output_dim= classifier.nb_class,
             dropout = 0.5)

In [174]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.01)


In [175]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 20,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 50)

| Epoch: 00 | Train Loss: 0.957 | Train Acc: 44.02%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.871560  0.174825  0.450658
recall      0.592516  0.316456  0.640187
accuracy    0.652555  0.722132  0.646889
f1-measure  0.705446  0.225225  0.528958
| Validation Accuracy : 57.75 % - Macro F1 Score : 0.4865 
| CM on the Validation : 
[[285.  72. 124.]
 [ 11.  25.  43.]
 [ 31.  46. 137.]]


| Epoch: 01 | Train Loss: 0.945 | Train Acc: 58.73%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.840541  0.144509  0.515152
recall      0.646570  0.316456  0.556075
accuracy    0.665205  0.692542  0.687311
f1-measure  0.730905  0.198413  0.534831
| Validation Accuracy : 58.79 % - Macro F1 Score : 0.4880 
| CM on the Validation : 
[[311.  92.  78.]
 [ 20.  25.  34.]
 [ 39.  56. 119.]]


| Epoch: 02 | Train Loss: 0.813 | Train Acc: 64.54%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.820896  

| Epoch: 17 | Train Loss: 0.330 | Train Acc: 90.96%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.812500  0.285714  0.563707
recall      0.810811  0.126582  0.682243
accuracy    0.751032  0.853125  0.751032
f1-measure  0.811655  0.175439  0.617336
| Validation Accuracy : 70.54 % - Macro F1 Score : 0.5348 
| CM on the Validation : 
[[390.  16.  75.]
 [ 31.  10.  38.]
 [ 59.   9. 146.]]


| Epoch: 18 | Train Loss: 0.357 | Train Acc: 91.99%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.817021  0.300000  0.554745
recall      0.798337  0.113924  0.710280
accuracy    0.748626  0.856918  0.747599
f1-measure  0.807571  0.165138  0.622951
| Validation Accuracy : 70.41 % - Macro F1 Score : 0.5319 
| CM on the Validation : 
[[384.  14.  83.]
 [ 31.   9.  39.]
 [ 55.   7. 152.]]


| Epoch: 19 | Train Loss: 0.240 | Train Acc: 92.64%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.810700  

In [176]:
# 5th Step : Test the Model and save data 
classifier.test(folder='CNN_paper')

| Successfully save to :  ../data/results/CNN_paper/subtask_c.csv


### 4) Simple RNN : 
Best Parameters :

In [77]:
# 2nd Step : Define Model : 
model = RNN(embedding= classifier.embedding,  
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=2,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [100])) #Change this to add/remove middle layers from output


In [78]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.0005)

In [79]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 20,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 300)

| Epoch: 00 | Train Loss: 1.091 | Train Acc: 36.80%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.663934  0.118881       NaN
recall      0.673597  0.430380  0.000000
accuracy    0.527246  0.546565  0.625874
f1-measure  0.668731  0.186301  0.000000
| Validation Accuracy : 46.25 % - Macro F1 Score : 0.2850 
| CM on the Validation : 
[[324. 157.   0.]
 [ 45.  34.   0.]
 [119.  95.   0.]]




/data/anaconda/envs/mli/lib/python3.6/site-packages/ipykernel_launcher.py:199: RuntimeWarning: invalid value encountered in double_scalars


| Epoch: 01 | Train Loss: 1.085 | Train Acc: 41.72%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.697460  0.129496  0.301587
recall      0.627859  0.455696  0.088785
accuracy    0.535232  0.556075  0.598993
f1-measure  0.660832  0.201681  0.137184
| Validation Accuracy : 46.12 % - Macro F1 Score : 0.3332 
| CM on the Validation : 
[[302. 142.  37.]
 [ 36.  36.   7.]
 [ 95. 100.  19.]]


| Epoch: 02 | Train Loss: 1.081 | Train Acc: 47.76%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.756757  0.129555  0.414013
recall      0.582121  0.405063  0.303738
accuracy    0.564371  0.589984  0.610032
f1-measure  0.658049  0.196319  0.350404
| Validation Accuracy : 48.71 % - Macro F1 Score : 0.4016 
| CM on the Validation : 
[[280. 125.  76.]
 [ 31.  32.  16.]
 [ 59.  90.  65.]]


| Epoch: 03 | Train Loss: 1.028 | Train Acc: 50.80%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.761671  

| Epoch: 18 | Train Loss: 0.504 | Train Acc: 83.85%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.853093  0.123810  0.483986
recall      0.688150  0.164557  0.635514
accuracy    0.698690  0.752351  0.682788
f1-measure  0.761795  0.141304  0.549495
| Validation Accuracy : 62.02 % - Macro F1 Score : 0.4842 
| CM on the Validation : 
[[331.  49. 101.]
 [ 22.  13.  44.]
 [ 35.  43. 136.]]


| Epoch: 19 | Train Loss: 0.317 | Train Acc: 85.39%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.867606  0.132812  0.477663
recall      0.640333  0.215190  0.649533
accuracy    0.678363  0.728414  0.671491
f1-measure  0.736842  0.164251  0.550495
| Validation Accuracy : 59.95 % - Macro F1 Score : 0.4839 
| CM on the Validation : 
[[308.  65. 108.]
 [ 18.  17.  44.]
 [ 29.  46. 139.]]




In [80]:
# 5th Step : Test the Model and save data 
classifier.test(folder='RNN')

| Successfully save to :  ../data/results/RNN/subtask_c.csv


### 5) LSTM : 
Best Parameters :

In [120]:
# 2nd Step : Define Model : 
model = LSTM(embedding= classifier.embedding,  
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=1,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [200,100])) #Change this to add/remove middle layers from output

In [121]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.0001)

In [122]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 10,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 300)

| Epoch: 00 | Train Loss: 1.006 | Train Acc: 41.32%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.889590  0.166667  0.424719
recall      0.586279  0.025316  0.883178
accuracy    0.669024  0.844643  0.627321
f1-measure  0.706767  0.043956  0.573596
| Validation Accuracy : 61.11 % - Macro F1 Score : 0.4414 
| CM on the Validation : 
[[282.   7. 192.]
 [ 13.   2.  64.]
 [ 22.   3. 189.]]


| Epoch: 01 | Train Loss: 0.997 | Train Acc: 52.53%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.869565  0.145078  0.552632
recall      0.706861  0.354430  0.490654
accuracy    0.711278  0.686502  0.709145
f1-measure  0.779817  0.205882  0.519802
| Validation Accuracy : 61.11 % - Macro F1 Score : 0.5018 
| CM on the Validation : 
[[340.  86.  55.]
 [ 21.  28.  30.]
 [ 30.  79. 105.]]


| Epoch: 02 | Train Loss: 0.865 | Train Acc: 58.83%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.907975  

In [ ]:
# 5th Step : Test the Model and save data 
classifier.test(folder='LSTM')

### 6) GRU : 
Best Parameters :

In [114]:
# 2nd Step : Define Model : 
model = GRU(embedding= classifier.embedding,  
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=1,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [200,100])) #Change this to add/remove middle layers from output

In [115]:
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.005)

In [116]:
# 4th Step : Train the Model
classifier.train( nb_epochs= 20,
                       optimizer= optimizer , 
                       model= model ,
                       batch_size= 100)

| Epoch: 00 | Train Loss: 0.996 | Train Acc: 40.81%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.825472       NaN  0.482857
recall      0.727651  0.000000  0.789720
accuracy    0.716851  0.867893  0.696644
f1-measure  0.773481  0.000000  0.599291
| Validation Accuracy : 67.05 % - Macro F1 Score : 0.4576 
| CM on the Validation : 
[[350.   0. 131.]
 [ 29.   0.  50.]
 [ 45.   0. 169.]]




/data/anaconda/envs/mli/lib/python3.6/site-packages/ipykernel_launcher.py:199: RuntimeWarning: invalid value encountered in double_scalars


| Epoch: 01 | Train Loss: 0.924 | Train Acc: 51.25%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.925373  0.170732  0.430108
recall      0.515593  0.088608  0.934579
accuracy    0.642655  0.811052  0.619891
f1-measure  0.662216  0.116667  0.589102
| Validation Accuracy : 58.79 % - Macro F1 Score : 0.4560 
| CM on the Validation : 
[[248.  29. 204.]
 [ 11.   7.  61.]
 [  9.   5. 200.]]


| Epoch: 02 | Train Loss: 0.923 | Train Acc: 53.16%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.911429  0.162162  0.520000
recall      0.663202  0.151899  0.850467
accuracy    0.726629  0.799065  0.719495
f1-measure  0.767750  0.156863  0.645390
| Validation Accuracy : 66.28 % - Macro F1 Score : 0.5233 
| CM on the Validation : 
[[319.  45. 117.]
 [ 16.  12.  51.]
 [ 15.  17. 182.]]


| Epoch: 03 | Train Loss: 0.921 | Train Acc: 54.51%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.890411  

| Epoch: 18 | Train Loss: 0.379 | Train Acc: 85.52%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.865079  0.117647  0.500000
recall      0.679834  0.151899  0.686916
accuracy    0.703329  0.755832  0.694286
f1-measure  0.761350  0.132597  0.578740
| Validation Accuracy : 62.79 % - Macro F1 Score : 0.4909 
| CM on the Validation : 
[[327.  52. 102.]
 [ 22.  12.  45.]
 [ 29.  38. 147.]]


| Epoch: 19 | Train Loss: 0.308 | Train Acc: 87.58%
| Stats on the Validation : 
                 IND       OTH       GRP
precision   0.879310  0.137615  0.482650
recall      0.636175  0.189873  0.714953
accuracy    0.685962  0.750000  0.678112
f1-measure  0.738239  0.159574  0.576271
| Validation Accuracy : 61.24 % - Macro F1 Score : 0.4914 
| CM on the Validation : 
[[306.  57. 118.]
 [ 18.  15.  46.]
 [ 24.  37. 153.]]




In [123]:
# 5th Step : Test the Model and save data 
classifier.test(folder='GRU')

| Successfully save to :  ../data/results/GRU/subtask_c_1.csv


### :O

In [134]:
classifierA = OffensiveClassifier(subtask='subtask_a', dim_vect=200 , pValid = 0.2)
classifierB = OffensiveClassifier(subtask='subtask_b', dim_vect=200 , pValid = 0.2)
classifierC = OffensiveClassifier(subtask='subtask_c', dim_vect=200 , pValid = 0.2)

classifiers = [classifierA , classifierB , classifierC]

---- Loading the processed GloVe files : Done
---- Creating the Pytorch Embedding Layer  : Done
-- Data Handling : 
---- Load the Clean Adapted Dataset : Done
---- Augmenting the Data : 
Before Augmentation :  {'NOT': 7072, 'OFF': 3520}
After Augmentation :  {'NOT': 7072, 'OFF': 7072}
---- Loading the processed GloVe files : Done
---- Creating the Pytorch Embedding Layer  : Done
-- Data Handling : 
---- Load the Clean Adapted Dataset : Done
---- Augmenting the Data : 
Before Augmentation :  {'UNT': 420, 'TIN': 3101}
After Augmentation :  {'UNT': 3101, 'TIN': 3101}
---- Loading the processed GloVe files : Done
---- Creating the Pytorch Embedding Layer  : Done
-- Data Handling : 
---- Load the Clean Adapted Dataset : Done
---- Augmenting the Data : 
Before Augmentation :  {'IND': 1926, 'OTH': 316, 'GRP': 860}
After Augmentation :  {'IND': 1926, 'OTH': 1926, 'GRP': 1926}


In [144]:
# 1st step : Define Classifier for specific task
# 3rd Step : Define Optinizer : 
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.005)

lrs=[1e-04,1e-03,1e-02]
weight_decays=[0,0.001,0.01]

model_RNN = RNN(embedding= classifier.embedding, 
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=1,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [])) 

model_LSTM = LSTM(embedding= classifier.embedding, 
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=1,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [])) 

model_GRU = GRU(embedding= classifier.embedding, 
            embedding_dim = classifier.dim_vect,
            hidden_size=classifier.dim_vect, 
            output_size=classifier.nb_class,
            num_layers=1,
            output_layer = output_layer(classifier.dim_vect, classifier.nb_class,
                middle_layers = [])) 

models = [model_RNN , model_LSTM , model_GRU]

middle_layers = [
    [],
    [200],
    [200,100],
]

#num_layers = [1]

hidden_sizes_extra = [-50,0,50]

results=[]
counter =0
for classifier in classifiers:
    for model in models:
        model.output_size = classifier.nb_class
        for hidden_size in hidden_sizes_extra:
            model.hidden_size = 200 + hidden_size
            for middle_layer in middle_layers:
                for lr in lrs:
                    for weight_decay in weight_decays:
                        optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
                        model.output_layer = output_layer(200 + hidden_size, classifier.nb_class,
                                                          middle_layers = middle_layer) 
                        score = classifier.train( nb_epochs= 10,
                               optimizer= optimizer , 
                               model= model ,
                               batch_size= 100)

                        if isinstance(model,RNN):
                            model_name="RNN"
                        elif isinstance(model,LSTM):
                            model_name="LSTM"
                        else:
                            model_name="GRU"
                        res={
                            "model": model_name,
                            "score": score,
                            "task": classifier.subtask,
                            "hidden_size": hidden_size,
                            "num_layer": num_layer,
                            "lr": lr,
                            "weight_decay": weight_decay,
                            "middle_layers": middle_layer
                        }
                        counter+=1
                        print(counter, res)
                        results.append(res)

| Epoch: 00 | Train Loss: 0.664 | Train Acc: 56.20%
| Stats on the Validation : 
                 NOT       OFF
precision   0.746373  0.478919
recall      0.727376  0.503409
accuracy    0.652946  0.652946
f1-measure  0.736752  0.490859
| Validation Accuracy : 65.29 % - Macro F1 Score : 0.6138 
| CM on the Validation : 
[[1286.  482.]
 [ 437.  443.]]


1 {'model': 'RNN', 'score': 0.6138051864271482, 'task': 'subtask_a', 'hidden_size': -50, 'num_layer': 2, 'lr': 0.0001, 'weight_decay': 0, 'middle_layers': []}
| Epoch: 00 | Train Loss: 0.661 | Train Acc: 62.18%
| Stats on the Validation : 
                 NOT       OFF
precision   0.778378  0.519837
recall      0.733032  0.580682
accuracy    0.682402  0.682402
f1-measure  0.755025  0.548578
| Validation Accuracy : 68.24 % - Macro F1 Score : 0.6518 
| CM on the Validation : 
[[1296.  472.]
 [ 369.  511.]]


2 {'model': 'RNN', 'score': 0.6518011613778616, 'task': 'subtask_a', 'hidden_size': -50, 'num_layer': 2, 'lr': 0.0001, 'weight_decay'

| Epoch: 00 | Train Loss: 0.524 | Train Acc: 76.30%
| Stats on the Validation : 
                 NOT       OFF
precision   0.867354  0.575859
recall      0.713801  0.780682
accuracy    0.736027  0.736027
f1-measure  0.783121  0.662808
| Validation Accuracy : 73.60 % - Macro F1 Score : 0.7230 
| CM on the Validation : 
[[1262.  506.]
 [ 193.  687.]]


15 {'model': 'RNN', 'score': 0.7229644204350694, 'task': 'subtask_a', 'hidden_size': -50, 'num_layer': 2, 'lr': 0.001, 'weight_decay': 0.01, 'middle_layers': [200]}
| Epoch: 00 | Train Loss: 0.436 | Train Acc: 64.35%
| Stats on the Validation : 
                 NOT       OFF
precision   0.836664  0.537176
recall      0.686652  0.730682
accuracy    0.701284  0.701284
f1-measure  0.754272  0.619162
| Validation Accuracy : 70.13 % - Macro F1 Score : 0.6867 
| CM on the Validation : 
[[1214.  554.]
 [ 237.  643.]]


16 {'model': 'RNN', 'score': 0.6867168830710411, 'task': 'subtask_a', 'hidden_size': -50, 'num_layer': 2, 'lr': 0.01, 'weight_d

| Epoch: 00 | Train Loss: 0.518 | Train Acc: 68.51%
| Stats on the Validation : 
                 NOT       OFF
precision   0.774974  0.605898
recall      0.833710  0.513636
accuracy    0.727341  0.727341
f1-measure  0.803270  0.555966
| Validation Accuracy : 72.73 % - Macro F1 Score : 0.6796 
| CM on the Validation : 
[[1474.  294.]
 [ 428.  452.]]


29 {'model': 'RNN', 'score': 0.6796176572119945, 'task': 'subtask_a', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.0001, 'weight_decay': 0.001, 'middle_layers': []}
| Epoch: 00 | Train Loss: 0.718 | Train Acc: 70.03%
| Stats on the Validation : 
                 NOT       OFF
precision   0.796023  0.586712
recall      0.792421  0.592045
accuracy    0.725831  0.725831
f1-measure  0.794218  0.589367
| Validation Accuracy : 72.58 % - Macro F1 Score : 0.6918 
| CM on the Validation : 
[[1401.  367.]
 [ 359.  521.]]


30 {'model': 'RNN', 'score': 0.691792101455967, 'task': 'subtask_a', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.0001, 'weight_decay

| Epoch: 00 | Train Loss: 0.629 | Train Acc: 62.55%
| Stats on the Validation : 
                 NOT       OFF
precision   0.788945  0.515152
recall      0.710407  0.618182
accuracy    0.679758  0.679758
f1-measure  0.747619  0.561983
| Validation Accuracy : 67.98 % - Macro F1 Score : 0.6548 
| CM on the Validation : 
[[1256.  512.]
 [ 336.  544.]]


43 {'model': 'RNN', 'score': 0.6548012593467138, 'task': 'subtask_a', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.01, 'weight_decay': 0, 'middle_layers': [200]}
| Epoch: 00 | Train Loss: 0.637 | Train Acc: 66.64%
| Stats on the Validation : 
                 NOT       OFF
precision   0.776645  0.643377
recall      0.861425  0.502273
accuracy    0.742069  0.742069
f1-measure  0.816841  0.564135
| Validation Accuracy : 74.21 % - Macro F1 Score : 0.6905 
| CM on the Validation : 
[[1523.  245.]
 [ 438.  442.]]


44 {'model': 'RNN', 'score': 0.690488133248382, 'task': 'subtask_a', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.01, 'weight_decay': 0.

| Epoch: 00 | Train Loss: 0.568 | Train Acc: 70.20%
| Stats on the Validation : 
                 NOT       OFF
precision   0.798378  0.577007
recall      0.779412  0.604545
accuracy    0.721299  0.721299
f1-measure  0.788781  0.590455
| Validation Accuracy : 72.13 % - Macro F1 Score : 0.6896 
| CM on the Validation : 
[[1378.  390.]
 [ 348.  532.]]


57 {'model': 'RNN', 'score': 0.6896179084868495, 'task': 'subtask_a', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.0001, 'weight_decay': 0.01, 'middle_layers': []}
| Epoch: 00 | Train Loss: 0.579 | Train Acc: 72.26%
| Stats on the Validation : 
                 NOT       OFF
precision   0.840847  0.546453
recall      0.696267  0.735227
accuracy    0.709215  0.709215
f1-measure  0.761757  0.626938
| Validation Accuracy : 70.92 % - Macro F1 Score : 0.6943 
| CM on the Validation : 
[[1231.  537.]
 [ 233.  647.]]


58 {'model': 'RNN', 'score': 0.6943477051193492, 'task': 'subtask_a', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.001, 'weight_deca

...

| Epoch: 00 | Train Loss: 0.276 | Train Acc: 96.13%
| Stats on the Validation : 
                 UNT       TIN
precision   0.179487  0.891076
recall      0.201923  0.876129
accuracy    0.796359  0.796359
f1-measure  0.190045  0.883539
| Validation Accuracy : 79.64 % - Macro F1 Score : 0.5368 
| CM on the Validation : 
[[ 21.  83.]
 [ 96. 679.]]


435 {'model': 'GRU', 'score': 0.5367923056315265, 'task': 'subtask_b', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.0001, 'weight_decay': 0.01, 'middle_layers': []}
| Epoch: 00 | Train Loss: 0.141 | Train Acc: 96.87%
| Stats on the Validation : 
                 UNT       TIN
precision   0.215190  0.891250
recall      0.163462  0.920000
accuracy    0.830489  0.830489
f1-measure  0.185792  0.905397
| Validation Accuracy : 83.05 % - Macro F1 Score : 0.5456 
| CM on the Validation : 
[[ 17.  87.]
 [ 62. 713.]]


436 {'model': 'GRU', 'score': 0.5455945875618007, 'task': 'subtask_b', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.001, 'weight_decay': 0, '

| Epoch: 00 | Train Loss: 0.001 | Train Acc: 95.73%
| Stats on the Validation : 
                 UNT       TIN
precision   0.163462  0.887742
recall      0.163462  0.887742
accuracy    0.802048  0.802048
f1-measure  0.163462  0.887742
| Validation Accuracy : 80.20 % - Macro F1 Score : 0.5256 
| CM on the Validation : 
[[ 17.  87.]
 [ 87. 688.]]


449 {'model': 'GRU', 'score': 0.5256017369727047, 'task': 'subtask_b', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.01, 'weight_decay': 0.001, 'middle_layers': [200]}
| Epoch: 00 | Train Loss: 0.162 | Train Acc: 93.71%
| Stats on the Validation : 
                 UNT       TIN
precision   0.171875  0.885890
recall      0.105769  0.931613
accuracy    0.833902  0.833902
f1-measure  0.130952  0.908176
| Validation Accuracy : 83.39 % - Macro F1 Score : 0.5196 
| CM on the Validation : 
[[ 11.  93.]
 [ 53. 722.]]


450 {'model': 'GRU', 'score': 0.5195642407906559, 'task': 'subtask_b', 'hidden_size': 0, 'num_layer': 2, 'lr': 0.01, 'weight_decay': 0.0

| Epoch: 00 | Train Loss: 0.012 | Train Acc: 98.05%
| Stats on the Validation : 
                 UNT       TIN
precision   0.180723  0.888191
recall      0.144231  0.912258
accuracy    0.821388  0.821388
f1-measure  0.160428  0.900064
| Validation Accuracy : 82.14 % - Macro F1 Score : 0.5302 
| CM on the Validation : 
[[ 15.  89.]
 [ 68. 707.]]


463 {'model': 'GRU', 'score': 0.530245730605187, 'task': 'subtask_b', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.001, 'weight_decay': 0, 'middle_layers': []}
| Epoch: 00 | Train Loss: 0.003 | Train Acc: 99.06%
| Stats on the Validation : 
                 UNT       TIN
precision   0.191919  0.891026
recall      0.182692  0.896774
accuracy    0.812287  0.812287
f1-measure  0.187192  0.893891
| Validation Accuracy : 81.23 % - Macro F1 Score : 0.5405 
| CM on the Validation : 
[[ 19.  85.]
 [ 80. 695.]]


464 {'model': 'GRU', 'score': 0.5405413967338792, 'task': 'subtask_b', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.001, 'weight_decay': 0.001, 

| Epoch: 00 | Train Loss: 0.223 | Train Acc: 95.03%
| Stats on the Validation : 
                 UNT       TIN
precision   0.208333  0.889715
recall      0.144231  0.926452
accuracy    0.833902  0.833902
f1-measure  0.170455  0.907712
| Validation Accuracy : 83.39 % - Macro F1 Score : 0.5391 
| CM on the Validation : 
[[ 15.  89.]
 [ 57. 718.]]


477 {'model': 'GRU', 'score': 0.5390831513619124, 'task': 'subtask_b', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.01, 'weight_decay': 0.01, 'middle_layers': [200]}
| Epoch: 00 | Train Loss: 1.700 | Train Acc: 95.44%
| Stats on the Validation : 
                 UNT       TIN
precision   0.210938  0.897470
recall      0.259615  0.869677
accuracy    0.797497  0.797497
f1-measure  0.232759  0.883355
| Validation Accuracy : 79.75 % - Macro F1 Score : 0.5581 
| CM on the Validation : 
[[ 27.  77.]
 [101. 674.]]


478 {'model': 'GRU', 'score': 0.5580568988114069, 'task': 'subtask_b', 'hidden_size': 50, 'num_layer': 2, 'lr': 0.0001, 'weight_decay': 

RuntimeError: CUDA error: device-side assert triggered

In [169]:
import pandas as pd

res_df = pd.DataFrame(results)
res_df.loc[res_df.task == "subtask_b"].loc[res_df.model == "GRU"].sort_values(by=["score"],ascending=[False]).iloc[:15]

,hidden_size,lr,middle_layers,model,num_layer,score,task,weight_decay
427,-50,0.0010,"[200, 100]",GRU,2,0.574421,subtask_b,0.001
467,50,0.0100,[],GRU,2,0.565834,subtask_b,0.010
457,0,0.0100,"[200, 100]",GRU,2,0.558140,subtask_b,0.001
477,50,0.0001,"[200, 100]",GRU,2,0.558057,subtask_b,0.000
417,-50,0.0010,[200],GRU,2,0.557426,subtask_b,0.000
419,-50,0.0010,[200],GRU,2,0.557362,subtask_b,0.010
426,-50,0.0010,"[200, 100]",GRU,2,0.554605,subtask_b,0.000
411,-50,0.0100,[],GRU,2,0.552327,subtask_b,0.000
437,0,0.0010,[],GRU,2,0.552075,subtask_b,0.010
441,0,0.0001,[200],GRU,2,0.550560,subtask_b,0.000
